In [87]:
import tensorflow as tf
import tensorlayer as tl
from tensorlayer.prepro import *
from PIL import Image
import os
import glob
import numpy as np

In [135]:
def convert_process(file_name,trainHRPath):
    im = Image.open(trainHRPath+"/"+file_name)
    im.save(trainHRPath+"/"+str(file_name[0:-3])+"png")
    return True

def downsample_fn(x,savePath):
    # We obtained the LR images by downsampling the HR images using bicubic kernel with downsampling factor r = 4.
    im = imresize(x[0], size=[int(np.floor(x[0].shape[0]/4)),int(np.floor(x[0].shape[1]/4))], interp='bicubic', mode=None)
    im = Image.fromarray(im)
    im.save(savePath+"/"+str(x[1][0:-3])+"png")
    return True

In [105]:
trainHRPath = "stage/train/HR"
testHRPath = "stage/test/HR"

In [106]:
trainHR = sorted(tl.files.load_file_list(path=trainHRPath, regx='.*.jpg', printable=False))
testHR = sorted(tl.files.load_file_list(path=testHRPath, regx='.*.jpg', printable=False))

In [107]:
train_hr_imgs = tl.vis.read_images(trainHR, path=trainHRPath, n_threads=32)

[TL] read 32 from stage/train/HR
[TL] read 64 from stage/train/HR
[TL] read 96 from stage/train/HR
[TL] read 128 from stage/train/HR
[TL] read 160 from stage/train/HR
[TL] read 192 from stage/train/HR
[TL] read 224 from stage/train/HR
[TL] read 256 from stage/train/HR
[TL] read 288 from stage/train/HR
[TL] read 300 from stage/train/HR


In [136]:
zippedPair = list(zip(train_hr_imgs,trainHR))
_ = tl.prepro.threading_data(zippedPair, fn=downsample_fn,savePath="stage/train/LR")

In [139]:
del train_hr_imgs,zippedPair

In [144]:
test_hr_imgs = tl.vis.read_images(testHR, path=testHRPath, n_threads=32)

[TL] read 32 from stage/test/HR
[TL] read 64 from stage/test/HR
[TL] read 96 from stage/test/HR
[TL] read 128 from stage/test/HR
[TL] read 160 from stage/test/HR
[TL] read 192 from stage/test/HR
[TL] read 200 from stage/test/HR


In [141]:
tl.files.exists_or_mkdir("stage/test/LR")

[TL] [*] creates stage/test/LR ...


False

In [142]:
zippedPair = list(zip(test_hr_imgs,testHR))
_ = tl.prepro.threading_data(zippedPair, fn=downsample_fn,savePath="stage/test/LR")

In [145]:
del test_hr_imgs,zippedPair

In [146]:
_ = tl.prepro.threading_data(trainHR, fn=convert_process,trainHRPath=trainHRPath)
_ = tl.prepro.threading_data(testHR, fn=convert_process,trainHRPath=testHRPath)

In [147]:
for fl in glob.glob(trainHRPath+"/*.jpg"):
    os.remove(fl)
for fl in glob.glob(testHRPath+"/*.jpg"):
    os.remove(fl)